In [1]:
import matplotlib.pyplot as plt;
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cluster import KMeans
from kneed import DataGenerator, KneeLocator
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [2]:
path=r''
traData = pd.read_csv(path+'trainingGroup.csv',index_col=None, header=None)
#traData = pd.read_csv('D:\\drive\\publications_2017\\datasets\\NSL-KDD\\NSL-KDD\\testingData.csv'  , index_col = None , header = None)
tesData = pd.read_csv(path+'testingGroup.csv',index_col=None, header=None)
target = 41

y_train = traData[[target]].values.reshape(len(traData)) 
x_train = preprocessing.normalize(traData.iloc[:,:-1])

y_test = tesData[[target]].values.reshape(len(tesData)) 
x_test = preprocessing.normalize(tesData.iloc[:,:-1])

In [3]:
def chooseObsMan(dic_f):
    chosenIDs=np.array(())
    for k,d in dic_f.items():
        temp=np.where(y_train==k)[0]
        z=np.random.choice(temp,dic_f[k],replace=False)
        chosenIDs=np.concatenate((chosenIDs,z))
    return np.int64(chosenIDs)
def label_index(row,threshold=0):
    
    if threshold==0:
        threshold=np.max(row)
    temp_i=np.where(row>=threshold)[0]
    if temp_i.shape[0]>0:
        return temp_i[0]+1
    else:
        return -1

In [4]:
labelled=np.zeros(x_train.shape[0])
labelled[:]=-1
li={1:60,2:60,3:30,4:10,5:10}
labelledIndex=chooseObsMan(li)
labelled[labelledIndex]=y_train[labelledIndex]

In [5]:
#method to find the nearest clusters :--- output is 5
def findBestclusters():
    kmeans_kwargs = { "init": "random", "n_init": 10, "max_iter": 300, "random_state": 42, }
    # A list holds the SSE values for each k
    sse = []
    for k in range(1, 25):
        kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
        kmeans.fit(x_train)
        sse.append(kmeans.inertia_)

    plt.style.use("fivethirtyeight")
    plt.plot(range(1, 25), sse)
    plt.xticks(range(1, 25))
    plt.xlabel("Number of Clusters")
    plt.ylabel("SSE")
    plt.show()

    kl = KneeLocator(range(1, 25), sse, curve="convex", direction="decreasing")
    return kl.elbow    #result is 5

#print(findBestclusters())

In [6]:
kmeans = KMeans( init="random", n_clusters=10, n_init=10, max_iter=300, random_state=42).fit(x_train)

In [7]:
# Final locations of the centroid
#print(kmeans.cluster_centers_)

# The number of iterations required to converge
#print(kmeans.n_iter_)

In [8]:
clusters = np.full((10, 6), 0, dtype=int)
for i in range(kmeans.labels_.shape[0]):
    clusters[ kmeans.labels_[i] ][ y_train[i] ] = clusters[ kmeans.labels_[i] ][ y_train[i] ] + 1

df = pd.DataFrame(clusters[:,1:11]
                    , index = ["Cluster0", "Cluster1", "Cluster2", "Cluster3", "Cluster4","Cluster5", "Cluster6", "Cluster7", "Cluster8", "Cluster9"]
                    , columns=['normal', 'DoS', 'Probe', 'R2L', 'U2R'])
print('total number of points in each cluster: \n')
print(df)
#print(clusters)
print('\nLABEL OF EACH CLUSTER :\ncluster0-->Probe\ncluster1-->DoS\ncluster2-->normal\ncluster3-->R2L\ncluster4-->Probe\ncluster5-->DoS\ncluster6-->U2R')
print('cluster7-->Probe\ncluster8-->normal\ncluster9-->normal')

total number of points in each cluster: 

          normal    DoS  Probe  R2L  U2R
Cluster0    2937     50   3720    7    0
Cluster1      19  16075    470    0    0
Cluster2   10630      0      8   51    2
Cluster3   11707   3526    251  582    0
Cluster4    2706   5054   3705    1    4
Cluster5      11  20670   1896    0    0
Cluster6   27259      0     34  313   46
Cluster7    1686      0   1021   32    0
Cluster8    3776    153    123    1    0
Cluster9    6612    399    428    8    0

LABEL OF EACH CLUSTER :
cluster0-->Probe
cluster1-->DoS
cluster2-->normal
cluster3-->R2L
cluster4-->Probe
cluster5-->DoS
cluster6-->U2R
cluster7-->Probe
cluster8-->normal
cluster9-->normal


In [9]:
#KNN model using Kmeans results
clustering_out = [3, 2, 1, 4, 3, 2, 5, 3, 1, 1]
clustering_out_label = ['Probe', 'DoS', 'normal', 'R2L', 'Probe', 'DoS', 'U2R', 'Probe', 'normal', 'normal']

In [10]:
knn_model = KNeighborsRegressor(n_neighbors=1)
knn_model.fit(kmeans.cluster_centers_, clustering_out)

KNeighborsRegressor(n_neighbors=1)

In [11]:
train_preds = knn_model.predict(x_train)
print((train_preds == y_train).mean())

0.5305660736824558


In [12]:
matrices = metrics.precision_recall_fscore_support(y_train, train_preds,labels=[1, 2, 3, 4, 5])
resultsDataFrame = pd.DataFrame()
for i in range(0,len(matrices)):
    temp = pd.DataFrame([matrices[i]],columns=['normal','DoS','Probe','R2L','U2R'])
    resultsDataFrame = resultsDataFrame.append(temp, ignore_index=True)
resultsDataFrame.insert(0,'  ',['Precision','Recall','Fscore','Support'])
print(resultsDataFrame)

correctLabelled=metrics.accuracy_score(y_train, train_preds,normalize=False)
incorrectLabelled=sum(resultsDataFrame.iloc[3,1:]) - correctLabelled
print([correctLabelled,incorrectLabelled])

numbers=resultsDataFrame.iloc[3,1:]
correctLabelledForEachclass=resultsDataFrame.iloc[3,1:] *resultsDataFrame.iloc[1,1:]
incorrectLabelledForEachclass=resultsDataFrame.iloc[3,1:]-correctLabelledForEachclass
results1 = pd.DataFrame(columns = ['Total','Correct', 'Incorrect'])
results1['Total'] = numbers
results1['Correct'] = correctLabelledForEachclass
results1['Incorrect'] = incorrectLabelledForEachclass
print(results1)

                    normal           DoS         Probe         R2L        U2R
0  Precision      0.947141      0.938785      0.403671    0.036226   0.001664
1     Recall      0.312104      0.800074      0.724605    0.584925   0.884615
2     Fscore      0.469498      0.863897      0.518494    0.068226   0.003321
3    Support  67343.000000  45927.000000  11656.000000  995.000000  52.000000
[66837, 59136.0]
        Total Correct Incorrect
normal  67343   21018     46325
DoS     45927   36745      9182
Probe   11656    8446      3210
R2L       995     582       413
U2R        52      46         6
